# Comparative Country Dashboard

The Comparative Country Dashboard provides an overview of countries along with historical analysis across economic and social indicators. This script retrieves data on key indicators from the [World Bank](https://www.worldbank.org/ext/en/home) and compiles them into a single dataframe, which is then utilized for developing a streamlit app.

In [3]:
# libraries
import requests
import pandas as pd

In [4]:
# function to fetch indicator data from the World Bank API
def fetch_data (indicator_code):
    
    # fetch country metadata to filter out regions
    country_metadata_url = "http://api.worldbank.org/v2/country"
    metadata_params = {"format": "json", "per_page": 300}  # fetch all available countries
    response_metadata = requests.get(country_metadata_url, params=metadata_params) # HTTP GET request to fetch country metadata

    if response_metadata.status_code == 200:
        country_metadata = response_metadata.json()[1]
        # list of country IDs, filtering out regional entries (where region ID is 'NA')
        valid_countries = [country["id"] for country in country_metadata if country["region"]["id"] != "NA"]
    else:
        raise Exception(f"Error fetching country metadata: {response_metadata.status_code}")

    # base URL for API
    base_url = "http://api.worldbank.org/v2"
    params = {
        "format": "json",
        "per_page": "30000", # set a high number to attempt fetching all data in one request
        "date": "2015:2024",
        "countries": ";".join(valid_countries)  # fetch only valid countries codes separated by semicolons for the API request
    }

    # fetch indicator data for the specified countries and indicator code
    response = requests.get(f"{base_url}/country/{params['countries']}/indicator/{indicator_code}", params=params)
    
    if response.status_code == 200:
        data = response.json()[1]  # extract relevant data
        df = pd.DataFrame(data) # convert raw data to a df
    else:
        raise Exception(f"Error fetching data for indicator {indicator_code}: {response.status_code}")
    
    # clean and process the data
    if not df.empty:
        df = df[['countryiso3code', 'date', 'value']].dropna() # select relevant columns and drop rows with NaN values
        df.rename(columns={'countryiso3code': 'Country', 'date': 'Year', 'value': indicator_code}, inplace=True) # rename columns
        df = df.pivot(index='Country', columns='Year', values=indicator_code) # pivot the df to have countries as rows and years as columns
    else:
        df = pd.DataFrame(columns=['Country', 'Year', indicator_code])  # return empty df if no data

    return df

In [6]:
# Total population
POP_code = "SP.POP.TOTL"
df_POP = fetch_data (POP_code)
df_POP.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,107906.0,108727.0,108735.0,108908.0,109203.0,108587.0,107700.0,107310.0,107359.0
AFG,33831764.0,34700612.0,35688935.0,36743039.0,37856121.0,39068979.0,40000412.0,40578842.0,41454761.0


### Economic Indicators

In [7]:
# GDP
GDP_code = "NY.GDP.MKTP.CD"
df_GDP = fetch_data (GDP_code)
df_GDP.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,2.962907e+09,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.481857e+09,2.929447e+09,3.279344e+09,3.648573e+09
AFG,1.913422e+10,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426000e+10,1.449724e+10,1.723305e+10


In [8]:
# GDP per capita
GDP_PCAP_code = "NY.GDP.PCAP.CD"
df_GDP_PCAP = fetch_data (GDP_PCAP_code)
df_GDP_PCAP.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,27458.225331,27441.529662,28440.051964,30082.127645,31096.205074,22855.932320,27200.061079,30559.533535,33984.790620
AFG,565.569730,522.082216,525.469771,491.337221,496.602504,510.787063,356.496214,357.261153,415.707417


In [9]:
# Purchasing Power Parity
PPP_code = "PA.NUS.PPP" 
df_PPP = fetch_data (PPP_code)
df_PPP.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,1.366314,1.360015,1.356637,1.370606,1.423212,1.411909,1.363954,1.313380,1.352821
AFG,15.823582,15.923656,15.420207,14.856235,15.026434,15.335243,14.587942,14.897971,14.806405


In [10]:
# Inflation Rate
IR_code = "FP.CPI.TOTL.ZG"
df_IR = fetch_data (IR_code)
df_IR.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN,NaN
AFG,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN,NaN


In [8]:
# Employment to Population Ratio
# indicator_EPR = "SL.EMP.TOTL.SP.ZS"

### Social Indicators

In [11]:
# Life expectancy
LE_code = "SP.DYN.LE00.IN"
df_LE = fetch_data (LE_code)
df_LE.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022
Country,,,,,,,,
ABW,75.683,75.617,75.903,76.072,76.248,75.723,74.626,74.992
AFG,62.659,63.136,63.016,63.081,63.565,62.575,61.982,62.879


In [12]:
# Poverty Headcount Ratio at $2.15 a Day
# it shows the percentage of the population living on less than $2.15 a day, adjusted for purchasing power parity

POV_code = "SI.POV.DDAY"
df_POV = fetch_data (POV_code)
df_POV.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
AGO,NaN,NaN,NaN,31.1,NaN,NaN,NaN,NaN,NaN
ALB,0.1,0.1,0.4,0.0,0.0,0.0,NaN,NaN,NaN


In [14]:
# Employment to Population Ratio, 15+ (%, ILO Modeled Estimate)
# it shows the proportion of a country's working-age population that is employed

EMP_code = "SL.EMP.TOTL.SP.ZS"
df_EMP = fetch_data (EMP_code)
df_EMP.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
AFG,42.924,42.472,42.021,40.493,38.965,36.711,36.263,32.333,31.268
AGO,63.656,63.317,63.028,62.793,62.642,62.550,64.304,64.447,64.492


In [15]:
# Gross National Income (GNI) Per Capita (Constant PPP)

GNI_code = "NY.GNP.PCAP.PP.KD"
df_GNI = fetch_data (GNI_code)
df_GNI.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,NaN,NaN,NaN,NaN,NaN,NaN,34420.754148,NaN,NaN
AFG,NaN,NaN,NaN,NaN,NaN,2790.543441,2157.111817,1987.654859,2002.197223


### Education indicators

In [16]:
# School Enrollment, Secondary (% Gross)
# it measures the total enrollment in secondary education, regardless of age, expressed as a percentage of the 
# population in the official age group for secondary education.

ENRR_code = "SE.SEC.ENRR"
df_ENRR = fetch_data (ENRR_code)
df_ENRR.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,NaN,NaN,NaN,137.537155,140.313614,132.90802,124.235588,136.21669,NaN
AFG,53.285141,53.50634,55.402149,57.039299,NaN,NaN,NaN,NaN,NaN


In [17]:
# Expected Years of Schooling

SCH_code = "SE.SCH.LIFE"
df_SCH = fetch_data (SCH_code)
df_SCH.head(2)

Year,2015,2016,2017,2018,2019
Country,,,,,
AFG,NaN,NaN,NaN,10.19755,NaN
ALB,15.0763,14.8046,14.81613,14.69562,14.81892


In [18]:
# Literacy rate, adult total (% of people ages 15 and above)
# this indicator tracks the percentage of the adult population that can read and write
LITR_code = "SE.ADT.LITR.ZS"
df_LITR = fetch_data (LITR_code)
df_LITR.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,NaN,NaN,NaN,NaN,NaN,97.989998,NaN,NaN,NaN
AFG,33.753841,NaN,NaN,NaN,NaN,NaN,37.0,NaN,NaN


In [19]:
# Expenditure on education (% of GDP)
# this reflects the total public expenditure on education as a percentage of a country's GDP
XPD_code = "SE.XPD.TOTL.GD.ZS"
df_XPD = fetch_data (XPD_code)
df_XPD.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,5.88827,5.49136,4.456181,4.548759,4.435043,NaN,NaN,NaN,NaN
AFG,3.25580,4.54397,4.343190,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Gross primary school completion rate (%)
# it shows the percentage of students who complete the final grade of primary school.
PRM_CMPT_code = "SE.PRM.CMPT.ZS"
df_PRM_CMPT = fetch_data (PRM_CMPT_code)
df_PRM_CMPT.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,NaN,NaN,NaN,98.097252,108.233528,103.185188,100.936768,95.01976,NaN
AFG,NaN,82.910461,87.253326,88.735519,86.175522,NaN,NaN,NaN,NaN


### Infraestructure indicators

In [21]:
# Urban Population (% of Total Population)
# it indicates the share of the population living in urban areas
URB_code = "SP.URB.TOTL.IN.ZS"
df_URB = fetch_data (URB_code)
df_URB.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,43.108,43.192,43.293,43.411,43.546,43.697,43.866,44.052,44.254
AFG,24.803,25.020,25.250,25.495,25.754,26.026,26.314,26.616,26.933


In [22]:
# Access to Electricity (% of Population)
ELC_ACCS_code = "EG.ELC.ACCS.ZS"
df_ELC_ACCS = fetch_data (ELC_ACCS_code)
df_ELC_ACCS.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022
Country,,,,,,,,
ABW,100.0,100.0,100.0,100.0,100.0,100.0,100.0,99.9
AFG,71.5,97.7,97.7,93.4,97.7,97.7,97.7,85.3


In [23]:
# Renewable Energy Consumption (% of Total Final Energy Consumption)
# indicates the share of renewable energy in total energy consumption
RNEW_code = "EG.FEC.RNEW.ZS"
df_RNEW = fetch_data (RNEW_code)
df_RNEW.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022
Country,,,,,,,,
ABW,6.7,7.0,6.8,8.6,8.3,9.1,8.8,8.8
AFG,17.7,20.2,19.5,18.3,18.9,18.2,20.0,20.0


In [24]:
# Individuals Using the Internet (% of Population)
NET_code = "IT.NET.USER.ZS"
df_NET = fetch_data (NET_code)
df_NET.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,88.6612,93.5425,97.17,NaN,NaN,NaN,NaN,NaN,NaN
AFG,8.2600,11.0000,13.50,16.8,17.6,18.4,NaN,NaN,NaN


In [25]:
# Mobile Cellular Subscriptions (per 100 People)
# measures the number of mobile cellular subscriptions per 100 people
CEL_code = "IT.CEL.SETS.P2"
df_CEL = fetch_data (CEL_code)
df_CEL.head(2)

Year,2015,2016,2017,2018,2019,2020,2021,2022
Country,,,,,,,,
ABW,135.242717,NaN,NaN,NaN,132.466508,132.288784,132.349628,NaN
AFG,58.391096,62.371096,67.136415,59.902648,59.783879,58.190214,56.554435,NaN


In [27]:
# Roads, Paved (% of Total Roads)
# measures the proportion of roads that are paved.
#ROD_code = "IS.ROD.PAVE.ZS"
#df_ROD = fetch_data (ROD_code)
#df_ROD.head(2)

### Join dfs

In [157]:
#df_POP

#df_GDP
#df_GDP_PCAP
#df_PPP
#df_IR

#df_LE
#df_POV
#df_EMP
#df_GNI

#df_ENRR
#df_SCH
#df_LITR
#df_XPD
#df_PRM_CMPT

#df_URB
#df_ELC_ACCS
#df_RNEW
#df_NET
#df_CEL

In [28]:
df_POP

Year,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,
ABW,107906.0,108727.0,108735.0,108908.0,109203.0,108587.0,107700.0,107310.0,107359.0
AFG,33831764.0,34700612.0,35688935.0,36743039.0,37856121.0,39068979.0,40000412.0,40578842.0,41454761.0
AGO,28157798.0,29183070.0,30234839.0,31297155.0,32375632.0,33451132.0,34532429.0,35635029.0,36749906.0
ALB,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,2837849.0,2811666.0,2777689.0,2745972.0
AND,72174.0,72181.0,73763.0,75162.0,76474.0,77380.0,78364.0,79705.0,80856.0
...,...,...,...,...,...,...,...,...,...
XKX,1788274.0,1777568.0,1791019.0,1797086.0,1788891.0,1790152.0,1786080.0,1768096.0,1756366.0
YEM,31159379.0,32109010.0,33090921.0,34085182.0,35111408.0,36134863.0,37140230.0,38222876.0,39390799.0
ZAF,56723537.0,57259551.0,57635162.0,58613001.0,59587885.0,60562381.0,61502603.0,62378410.0,63212384.0


In [29]:
# melt each df so that 'Year' becomes a column and the respective values are in rows

# For df_POP
df_POP = df_POP.reset_index()  # reset index to make 'Country' a column
df_POP = df_POP.melt(id_vars=['Country'], var_name='Year', value_name='Population')

In [31]:
###### Economic Indicators
df_GDP = df_GDP.reset_index()
df_GDP = df_GDP.melt(id_vars=['Country'], var_name='Year', value_name='GDP')

df_GDP_PCAP = df_GDP_PCAP.reset_index() 
df_GDP_PCAP = df_GDP_PCAP.melt(id_vars=['Country'], var_name='Year', value_name='GDP Per Capita')

df_PPP = df_PPP.reset_index()
df_PPP = df_PPP.melt(id_vars=['Country'], var_name='Year', value_name='PPP')

df_IR = df_IR.reset_index()
df_IR = df_IR.melt(id_vars=['Country'], var_name='Year', value_name='Inflation Rate')

In [32]:
###### Social Indicators
df_LE = df_LE.reset_index()
df_LE = df_LE.melt(id_vars=['Country'], var_name='Year', value_name='Life Exp.')

df_POV = df_POV.reset_index()
df_POV = df_POV.melt(id_vars=['Country'], var_name='Year', value_name='Poverty at $2.15 a Day')

df_EMP = df_EMP.reset_index()
df_EMP = df_EMP.melt(id_vars=['Country'], var_name='Year', value_name='Employment to Population Ratio')

df_GNI = df_GNI.reset_index()
df_GNI = df_GNI.melt(id_vars=['Country'], var_name='Year', value_name='Gross National Income Per Capita')

In [33]:
###### Education Indicators
df_ENRR = df_ENRR.reset_index()
df_ENRR = df_ENRR.melt(id_vars=['Country'], var_name='Year', value_name='School Enrollment')

df_SCH = df_SCH.reset_index()
df_SCH = df_SCH.melt(id_vars=['Country'], var_name='Year', value_name='Expected Years of Schooling')

df_LITR = df_LITR.reset_index()
df_LITR = df_LITR.melt(id_vars=['Country'], var_name='Year', value_name='Literacy rate')

df_XPD = df_XPD.reset_index()
df_XPD = df_XPD.melt(id_vars=['Country'], var_name='Year', value_name='Expenditure on education')

df_PRM_CMPT = df_PRM_CMPT.reset_index()
df_PRM_CMPT = df_PRM_CMPT.melt(id_vars=['Country'], var_name='Year', value_name='Gross primary school completion rate')

In [34]:
###### Infra Indicators
df_URB = df_URB.reset_index()
df_URB = df_URB.melt(id_vars=['Country'], var_name='Year', value_name='Urban Population')

df_ELC_ACCS = df_ELC_ACCS.reset_index()
df_ELC_ACCS = df_ELC_ACCS.melt(id_vars=['Country'], var_name='Year', value_name='Access to Electricity')

df_RNEW = df_RNEW.reset_index()
df_RNEW = df_RNEW.melt(id_vars=['Country'], var_name='Year', value_name='Renewable Energy Consumption')

df_NET = df_NET.reset_index()
df_NET = df_NET.melt(id_vars=['Country'], var_name='Year', value_name='Individuals Using the Internet')

df_CEL = df_CEL.reset_index()
df_CEL = df_CEL.melt(id_vars=['Country'], var_name='Year', value_name='Mobile Cellular Subscriptions')

In [43]:
# merge all DataFrames on 'Country' and 'Year'
df_annualIndicators = (
    df_POP.merge(df_GDP, on=['Country', 'Year'], how='outer')
          .merge(df_GDP_PCAP, on=['Country', 'Year'], how='outer')
          .merge(df_PPP, on=['Country', 'Year'], how='outer')
          .merge(df_IR, on=['Country', 'Year'], how='outer')
          .merge(df_LE, on=['Country', 'Year'], how='outer')
          .merge(df_POV, on=['Country', 'Year'], how='outer')
          .merge(df_EMP, on=['Country', 'Year'], how='outer')
          .merge(df_GNI, on=['Country', 'Year'], how='outer')
          .merge(df_ENRR, on=['Country', 'Year'], how='outer')
          .merge(df_SCH, on=['Country', 'Year'], how='outer')
          .merge(df_LITR, on=['Country', 'Year'], how='outer')
          .merge(df_XPD, on=['Country', 'Year'], how='outer')
          .merge(df_PRM_CMPT, on=['Country', 'Year'], how='outer')
          .merge(df_URB, on=['Country', 'Year'], how='outer')
          .merge(df_ELC_ACCS, on=['Country', 'Year'], how='outer')
          .merge(df_RNEW, on=['Country', 'Year'], how='outer')
          .merge(df_NET, on=['Country', 'Year'], how='outer')
          .merge(df_CEL, on=['Country', 'Year'], how='outer')
)

In [44]:
df_annualIndicators.head(2)

,Country,Year,Population,GDP,GDP Per Capita,PPP,Inflation Rate,Life Exp.,Poverty at $2.15 a Day,Employment to Population Ratio,...,School Enrollment,Expected Years of Schooling,Literacy rate,Expenditure on education,Gross primary school completion rate,Urban Population,Access to Electricity,Renewable Energy Consumption,Individuals Using the Internet,Mobile Cellular Subscriptions
0,ABW,2015,107906.0,2.962907e+09,27458.225331,1.366314,0.474764,75.683,NaN,NaN,...,NaN,NaN,NaN,5.88827,NaN,43.108,100.0,6.7,88.6612,135.242717
1,ABW,2016,108727.0,2.983635e+09,27441.529662,1.360015,-0.931196,75.617,NaN,NaN,...,NaN,NaN,NaN,5.49136,NaN,43.192,100.0,7.0,93.5425,NaN


In [40]:
import pycountry

# function to map country codes to names
def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_3=code).name
    except:
        return code  # return the code if name not found

In [45]:
df_annualIndicators['Country'] = df_annualIndicatorData['Country'].apply(get_country_name)
df_annualIndicators.head(2)

,Country,Year,Population,GDP,GDP Per Capita,PPP,Inflation Rate,Life Exp.,Poverty at $2.15 a Day,Employment to Population Ratio,...,School Enrollment,Expected Years of Schooling,Literacy rate,Expenditure on education,Gross primary school completion rate,Urban Population,Access to Electricity,Renewable Energy Consumption,Individuals Using the Internet,Mobile Cellular Subscriptions
0,Aruba,2015,107906.0,2.962907e+09,27458.225331,1.366314,0.474764,75.683,NaN,NaN,...,NaN,NaN,NaN,5.88827,NaN,43.108,100.0,6.7,88.6612,135.242717
1,Aruba,2016,108727.0,2.983635e+09,27441.529662,1.360015,-0.931196,75.617,NaN,NaN,...,NaN,NaN,NaN,5.49136,NaN,43.192,100.0,7.0,93.5425,NaN


In [46]:
# save df_annualIndicators
df_annualIndicators.to_csv('/Users/isabelafranceschi/Desktop/Git/ComparativeCountryDashboard/df_annualIndicators.csv')